In [1]:
import pandas as pd
import tweepy
import json
from MyOwnTools.keys import twitter


In [2]:
#create connections and API object for further use 
consumer_key = twitter['consumer_key']
consumer_secret = twitter['consumer_secret']
access_token = twitter['access_token']
access_secret = twitter['access_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

#pass auth and set the True values for the time-limit parameters of API-class object
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [3]:
api

## Get tweets from search results

In [53]:
#Set a term for search. Here double quotes for restriction to search exact phrase
search_words = '"Business Intelligence", -filter:retweets' # ' -filter:retweets' says for it's self

In [54]:
    # Get search results
#Set te start date to search
#date_since = '2019-03-20'
number_of_tweets = 80
tweets = tweepy.Cursor(api.search,
              q = search_words,
              lang = "en",
              tweet_mode = 'extended'
              ).items(number_of_tweets)# method '.items' for set the number of tweeets we want to get
              #geocode="41.766823,72.631776"

In [55]:
#check object created
tweets

## Save data to the file

In [56]:
#itterate over tweets and assign value to statuses in JSON format
status = [x._json for x in tweets]


In [57]:
#get a glance of data
status[1]['user']['id']

54317231

**Create new data frame with desired columns names**<br>
Get values of specified keys from each status

In [58]:
# set a columns names
columns = ['user_id', 'user_name', 'lang', 'followers', 'tweets']


In [59]:
#user id
ud = [status[i]['user']['id'] for i in range (number_of_tweets)]

In [60]:
#screen name
un = [status[i]['user']['screen_name'] for i in range (number_of_tweets)]

In [61]:
# language
lg = [status[i]['user']['lang'] for i in range (number_of_tweets)]

In [62]:
#number of followers
fw = [status[i]['user']['followers_count'] for i in range (number_of_tweets)]

In [63]:
#number of statuses in timeline
tw = [status[i]['user']['statuses_count'] for i in range (number_of_tweets)]

In [64]:
#create pandas dataframe object
df = pd.DataFrame({'user_id' : ud, 'user_name' : un, 'lang' : lg, 'followers' : fw, 'tweets' : tw}, columns = columns)

In [65]:
df.head()

,user_id,user_name,lang,followers,tweets
0,824786651964547072,AnalyticMusings,en,17,712
1,54317231,arjenvanberkum,en,36055,40756
2,2656729140,Phocas_Nico,en,43,339
3,26057123,PhocasSoftware,en,1702,1727
4,2242131800,steveparry2013,en,63,2229


In [66]:
df[(df['followers'] < 500) & (df['tweets']<1000)].drop_duplicates().shape[0]

24

In [ ]:
#save data to the file csv
#df.to_csv('df.csv', index = False)

**Get number of followers of specified user_id**

In [ ]:
#df['followers'] = 1
#for i in df.index:
  #  user = api.get_user(user_id = df.loc[i, 'user_id'])
   # df.loc[i,'followers'] = user._json['followers_count']

In [ ]:
#df['followers'] = [api.get_user(user_id = df.loc[i, 'user_id'])._json['followers_count'] for i in df.index]

In [ ]:
#df['statuses'] = [api.get_user(user_id = df.loc[i, 'user_id'])._json['statuses_count'] for i in df.index]

In [ ]:
#check object
#user._json